# Книжный червь

Компания купила крупный сервис для чтения книг по подписке. В базе данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель исследования:**
Проанализировать базу данных крупного сервиса для чтения книг по подписке.

**Ход исследования:**  
В моем распоряжении база данных с таблицами:
- `books` содержит данные о книгах;
- `authors` содержит данные об авторах;
- `publishers` cодержит данные об издательствах;
- `ratings` cодержит данные о пользовательских оценках книг;
- `reviews` cодержит данные о пользовательских обзорах на книги.

В связи с поставленной целью должны быть решены ряд задач:
1. Исследование таблиц.
2. Посчитать сколько книг вышло после 1 января 2000 года.
3. Для каждой книги посчитать количество обзоров и среднюю оценку.
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — исключить из анализа брошюры.
5. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками.
6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.  
7. Общий вывод.

## Исследование таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': '', # имя пользователя
'pwd': '', # пароль
'host': '',
'port': , # порт подключения
'db': ''} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# напишем функцию для выполнения SQL-запроса, используя Pandas
def function_sql(query):
    display(pd.io.sql.read_sql(sql=text(query), con = engine))

In [2]:
function_sql('''SELECT * FROM books LIMIT 5''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
function_sql('''SELECT * FROM authors LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
function_sql('''SELECT * FROM publishers LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
function_sql('''SELECT * FROM ratings LIMIT 5''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
function_sql('''SELECT * FROM reviews LIMIT 5''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Cколько книг вышло после 1 января 2000 года

In [7]:
function_sql('''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01'
    ''')

,count
0,819


In [8]:
function_sql('''
    SELECT COUNT(book_id)
    FROM books
    WHERE          publication_date >= '2000-01-01'
    ''')

,count
0,821


**Вывод:** Отвечая на вопрос сколько книг вышло после 1 января 2000 года, можно сказать, что 1 января не включается, а значит вышло 819 книг.  
Однако довольно нетипично исключать 1 января и возможно стоит уточнить задачу, с 1 января 2000 года вышла 821 книга.

## Количество обзоров и средняя оценка по каждой книге

In [9]:
# в процессе исследования также обнаружены разные книги (судя по идентификатору), но с одинаковым названием
function_sql('''
    SELECT title,
           COUNT(DISTINCT book_id)
    FROM books
    GROUP BY 1
    HAVING COUNT(DISTINCT book_id) > 1
    ORDER BY 2 DESC
    ''') 

,title,count
0,Memoirs of a Geisha,2


In [10]:
function_sql('''
    WITH ra AS
        (SELECT book_id,
                AVG(rating) AS avg_rating
         FROM ratings
         GROUP BY 1
         ORDER BY 2 DESC),
    re as
        (SELECT book_id,
                COUNT(review_id) AS reviews_count
         FROM reviews
         GROUP BY 1
         ORDER BY 2 DESC)
         
    SELECT b.book_id,
           b.title,
           CASE WHEN re.reviews_count IS NULL THEN 0 ELSE re.reviews_count END,
           ROUND(ra.avg_rating, 2) as avg_rating
    FROM books as b
    FULL OUTER JOIN ra ON b.book_id=ra.book_id
    FULL OUTER JOIN re ON b.book_id=re.book_id
    ''') 

,book_id,title,reviews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,656,The Book Thief,6,4.26
2,497,Outlander (Outlander #1),6,4.13
3,963,Water for Elephants,6,3.98
4,696,The Da Vinci Code (Robert Langdon #2),6,3.83
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
997,221,Essential Tales and Poems,0,4.00
998,387,Leonardo's Notebooks,0,4.00


**Вывод:** Сформирована таблица с количеством обзоров и средней оценкой по каждой книге. Больше всего обзоров (7) у книги 'Twilight (Twilight #1)', рейтинг не так высок - 3,66. Среди книг с 6 обзорами самый высокий рейтинг (4,26) получила 'The Book Thief'. 

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
function_sql('''
    SELECT p.publisher,
           COUNT(b.book_id) AS books_count
    FROM books as b
    JOIN publishers as p ON b.publisher_id=p.publisher_id
    WHERE num_pages > 50
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
    ''')

,publisher,books_count
0,Penguin Books,42


**Вывод:** Больше всего книг выпустило издательство `Penguin Books.`  
Стоит отметить, что из анализа исключены брошюры, т.е. книги толщиной 50 страниц и менее.

## Автор с самой высокой средней оценкой книг

In [12]:
function_sql('''
    WITH x AS
        (SELECT b.book_id,
                COUNT(ra.rating_id)
        FROM books as b
        JOIN ratings as ra ON b.book_id=ra.book_id
        GROUP BY 1
        HAVING COUNT(ra.rating_id) >= 50)
        
    SELECT a.author,
           ROUND(AVG(ra.rating),2) as avg_rating
    FROM books as b
    JOIN ratings as ra ON b.book_id=ra.book_id
    JOIN author as a ON a.author_id=b.author_id
    WHERE b.book_id IN (SELECT book_id
                        FROM x)
    GROUP BY 1
    ORDER BY 2 DESC 
    LIMIT 1
    ''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** Автор с самой высокой средней оценкой книг - `J.K. Rowling/Mary GrandPré`. При анализе были учтены только книги с 50 и более оценками.

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
function_sql('''
    WITH x AS
        (SELECT username,
              COUNT(rating_id)
        FROM ratings
        GROUP BY 1
        HAVING COUNT(rating_id) > 50),
        
    rc AS
        (SELECT username,
           COUNT(review_id) review_count
        FROM reviews
        WHERE username IN (SELECT username
                           FROM x)
        GROUP BY 1)
    
    SELECT ROUND(AVG(review_count))
    FROM rc
    ''')

,round
0,24.0


**Вывод:** Пользователь, который поставил больше 50 оценок в среднем делает 24 обзора на книги.

## Общий вывод

В результате выполненного исследования мы проанализировали базу данных крупного сервиса для чтения книг по подписке и ответили на ряд вопросов.

1. Начиная с 1 января 2000 года вышла 821 книга.
2. Сформирована таблица с количеством обзоров и средней оценкой по каждой книге. Больше всего обзоров (7) у книги `Twilight (Twilight #1)`, рейтинг не так высок - 3,66. Среди книг с 6 обзорами самый высокий рейтинг (4,26) получила `The Book Thief`. 
3. Больше всего книг выпустило издательство `Penguin Books`. Стоит отметить, что из анализа исключены брошюры, т.е. книги толщиной 50 страниц и менее.
4. Автор с самой высокой средней оценкой книг - `J.K. Rowling/Mary GrandPré`. При анализе были учтены только книги с 50 и более оценками.
5. Пользователь, который поставил больше 50 оценок в среднем делает `24 обзора` на книги.
